In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT = 480

ANSWERED_CALLS = 1000
FINISHED_T = 0
all_customers = []
operation_t = [0,0]
shift_number = 0

In [3]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range)
        operation_t[1] += time
        yield env.timeout(time)

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.waiting_t = 0
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        
        
        # Voice recognition and routing
        if router.is_idle():
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        yield self.env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router.count.remove(self.name)
        if router.is_idle():
            router.idle_t += [self.env.now]
        
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            #all_customers.append(self)
            self.operator = None
            if self.name == f'Customer {ANSWERED_CALLS}':
                finished.succeed()
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
            self.waiting_t = self.env.now
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                #all_customers.append(self)
                self.operator = None
                if self.name == f'Customer {ANSWERED_CALLS}':
                    finished.succeed()
                return 
            self.waiting_t = self.env.now - self.waiting_t
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))
            #all_customers.append(self)
            if self.name == f'Customer {ANSWERED_CALLS}':
                finished.succeed()

In [5]:
class robocall:
    def __init__(self,env):
        self.count = []
        self.working_t = []
        self.idle_t = [0]
        self.env = env
        
    def answer(self):
        return len(self.count) < 100
    
    def is_idle(self):
        return len(self.count) == 0
            
    def utilization(self):
        return sum([ w-i for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

In [6]:
def breaks():
    return [random.uniform(0,SHIFT) for i in  range(np.random.poisson(8))]

In [7]:
def shift(env):
    global shift_number
    shift_number += 1
    
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)
    print("the time %g" % env.now)

    env.process(shift(env))

In [8]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
            print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [9]:
def customer_generator(env):
    start = True
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            all_customers.append(customer)
            
    FINISHED_T = env.now 

In [10]:
env = simpy.Environment()

finished = Event(env)
router = robocall(env)

operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

env.process(customer_generator(env))
env.run(finished) 
FINISHED_T = env.now
print(FINISHED_T)
print(len(all_customers))

Customer 1 initiated a call at 26.5136
break started for operator 1 at 34.076
Customer 2 initiated a call at 34.3427
break ended for operator 1 at 37.076
Customer 3 initiated a call at 42.7483
Customer 2 is rooted to operator 1 at 43.7922
Customer 2 is assigned to the operator 1 at 43.7922
Customer 1 is rooted to operator 2 at 44.769
Customer 1 is assigned to the operator 2 at 44.769
Customer 3 hangs up the call at 45.205
Customer 1 is done with operator 2 at 50.4135
Customer 2 is done with operator 1 at 53.9481
break started for operator 1 at 53.9481
Customer 4 initiated a call at 54.4264
Customer 4 is rooted to operator 2 at 54.9592
Customer 4 is assigned to the operator 2 at 54.9592
break ended for operator 1 at 56.9481
Customer 5 initiated a call at 59.1688
Customer 5 is rooted to operator 2 at 59.6663
Customer 4 is done with operator 2 at 61.136
Customer 5 is assigned to the operator 2 at 61.136
Customer 5 is done with operator 2 at 66.5906
Customer 6 initiated a call at 69.657
Cu

Customer 341 is assigned to the operator 2 at 2116.71
Customer 342 initiated a call at 2117.15
Customer 342 is rooted to operator 1 at 2119.32
Customer 342 is assigned to the operator 1 at 2119.32
Customer 341 is done with operator 2 at 2119.54
Customer 340 is assigned to the operator 2 at 2119.54
Customer 340 is done with operator 2 at 2122.18
Customer 343 initiated a call at 2126.96
Customer 342 is done with operator 1 at 2130.3
Customer 343 is rooted to operator 2 at 2132.02
Customer 343 is assigned to the operator 2 at 2132.02
break started for operator 1 at 2132.39
Customer 343 is done with operator 2 at 2135.28
break ended for operator 1 at 2135.39
Customer 344 initiated a call at 2141.36
Customer 345 initiated a call at 2141.97
Customer 344 is rooted to operator 2 at 2143.79
Customer 344 is assigned to the operator 2 at 2143.79
Customer 344 is done with operator 2 at 2146.13
Customer 345 is rooted to operator 2 at 2149.15
Customer 345 is assigned to the operator 2 at 2149.15
Cus

Customer 639 is rooted to operator 1 at 3916.39
Customer 639 is assigned to the operator 1 at 3916.39
Customer 638 is rooted to operator 2 at 3918.85
Customer 638 is assigned to the operator 2 at 3918.85
Customer 635 is rooted to operator 2 at 3925.28
Customer 638 is done with operator 2 at 3925.3
Customer 635 is assigned to the operator 2 at 3925.3
Customer 639 is done with operator 1 at 3927.28
Customer 635 is done with operator 2 at 3927.46
Customer 636 is rooted to operator 2 at 3927.71
Customer 636 is assigned to the operator 2 at 3927.71
Customer 636 is done with operator 2 at 3932.86
break started for operator 2 at 3932.86
break ended for operator 2 at 3935.86
Customer 640 initiated a call at 3948.39
Customer 640 is rooted to operator 1 at 3952.03
Customer 640 is assigned to the operator 1 at 3952.03
Customer 641 initiated a call at 3957.64
Customer 641 is rooted to operator 2 at 3959.12
Customer 641 is assigned to the operator 2 at 3959.12
Customer 642 initiated a call at 3960.

Customer 940 is left the operator 1 after 10 minutes at 5580.93
Customer 941 is done with operator 1 at 5581.96
Customer 947 is assigned to the operator 1 at 5581.96
Customer 949 initiated a call at 5583.23
Customer 950 initiated a call at 5583.38
Customer 949 is rooted to operator 2 at 5583.46
Customer 949 is assigned to the operator 2 at 5583.46
Customer 951 initiated a call at 5585.69
Customer 950 is rooted to operator 2 at 5586.64
Customer 949 is done with operator 2 at 5588.1
Customer 950 is assigned to the operator 2 at 5588.1
Customer 951 is rooted to operator 2 at 5588.84
Customer 950 is done with operator 2 at 5591.16
Customer 951 is assigned to the operator 2 at 5591.16
Customer 951 is done with operator 2 at 5594.68
Customer 947 is done with operator 1 at 5595.11
Customer 952 initiated a call at 5605.23
Customer 953 initiated a call at 5606.33
Customer 954 initiated a call at 5606.7
Customer 953 is rooted to operator 2 at 5607.24
Customer 953 is assigned to the operator 2 at

In [11]:
print('Voice Recognition System Utilization: ', router.utilization())

Voice Recognition System Utilization:  0.4383443621438291


In [12]:
print('Utilization Operator 1:', operation_t[0]/FINISHED_T, 
      '\nUtilization Operator 2:' ,operation_t[1]/FINISHED_T)

Utilization Operator 1: 0.43153429917961744 
Utilization Operator 2: 0.4266010803889242


In [13]:
customer_waiting = [c.waiting_t for c in all_customers if c.operator != None]
print('Average Waiting Time: ', sum(customer_waiting)/len(customer_waiting))

Average Waiting Time:  1.6219402559790854
